# pewpewpew

#### Importing libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import json
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import time
import random

# Part 1 - Collecting data about Moscow

In [2]:
# Let's ccreate empty DataFrame to fill it with data in future
df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)

# Let's get Okrug names at first
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["addr:country"="RU"]["addr:region"="Москва"][admin_level=5];
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

Okrug_Names = []
for i in range(len(data['elements'])):
    Okrug_Names.append(data['elements'][i]['tags']['name:en'])

df['Okrug_Name'] = Okrug_Names
df

,Okrug_Name
0,Northern Administrative Okrug
1,Western Administrative Okrug
2,North-Western Administrative Okrug
3,North-Eastern Administrative Okrug
4,South-Eastern Administrative Okrug
5,Southern Administrative Okrug
6,South-Western Administrative Okrug
7,Eastern Administrative Okrug
8,Zelenogradsky Administrative Okrug
9,Central Administrative Okrug


#### We will use geopy library to get the latitude and longitude values.
Let's write a function and name user agent as Moscow_explorer

In [3]:
def get_coord(addr):
    address = '%s' % addr
    geolocator = Nominatim(user_agent="Moscow_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude, longitude

#### Let's get Okrugs's coordinates and draw them on map

Getting coordinates of Moscow

In [8]:
Moscow_latitude, Moscow_longitude = get_coord('Moscow')
print('The geograpical coordinates of Moscow are {}, {}.'.format(Moscow_latitude, Moscow_longitude))
#The geograpical coordinates of Moscow are 55.7504461, 37.6174943.

The geograpical coordinates of Moscow are 55.7504461, 37.6174943.


Getting coordinates of every Okrug

In [9]:
# Okrug_lat = []
# Okrug_lon = []
# for Okrug in df['Okrug_Name']:
#     latitude, longitude = get_coord(Okrug)
#     Okrug_lat.append(latitude)
#     Okrug_lon.append(longitude)
#     time.sleep(1)
# df['Okrug_lat'] = Okrug_lat
# df['Okrug_lon'] = Okrug_lon

# # Save df to file
# df.to_csv('coords.csv', sep=',', encoding='utf-8', index=False)

# df

In [10]:
df = pd.read_csv('coords.csv')

In [11]:
# Let's draw map with Okrugs
map_Moscow = folium.Map(location=[Moscow_latitude, Moscow_longitude], zoom_start=9)
for Okrug_name, lat, lng in zip(df['Okrug_Name'], \
                             df['Okrug_lat'], \
                             df['Okrug_lon']):
    label = '{}'.format(Okrug_name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#863100',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Moscow)  
map_Moscow

In [12]:
# # Let's get districts of each Okrug
# Okrug_districts = []
# Okrug_dict = {}
# for Okrug in df['Okrug_Name']:
#     print('Processing', Okrug)
#     overpass_url = "http://overpass-api.de/api/interpreter"
#     overpass_query = """
#     [out:json];
#     area['name:en' = '%s'][type="boundary"];
#     rel(area)[admin_level=8][boundary=administrative];
#     out;
#     """ % Okrug
#     response = requests.get(overpass_url, 
#                             params={'data': overpass_query})
#     data = response.json()
#     Okrug_districts = []
#     for i in range(len(data['elements'])):
#         try:
#             Okrug_districts.append(data['elements'][i]['tags']['name:en'])
#         except:
#             Okrug_districts.append(data['elements'][i]['tags']['name'])
#     Okrug_dict[Okrug] = Okrug_districts
#     time.sleep(1)

In [13]:
# full = []
# for el in Okrug_dict.keys():
#     for el2 in Okrug_dict[el]:
#         full.append([el, el2])
# dff = pd.DataFrame(data=full, index=None, columns=['Okrug', 'District'], dtype=None, copy=False)
# dff.head()

In [14]:
# full_coords_lat = []
# full_coords_lon = []
# for index, row in dff.iterrows():
#     print('Processing', row['District'], 'of', row['Okrug'])
#     lat, lon = get_coord(','.join([row['Okrug'], row['District']]))
#     full_coords_lat.append(lat)
#     full_coords_lon.append(lon)
# dff['Latitude'] = full_coords_lat
# dff['Longitude'] = full_coords_lon

# # Save df to file
# dff.to_csv('coords_full.csv', sep=',', encoding='utf-8', index=False)

In [15]:
# Save df to file
dff = pd.read_csv('coords_full.csv')
dff.head()

,Okrug,District,Latitude,Longitude
0,Northern Administrative Okrug,Levoberezhny District,55.865663,37.465859
1,Northern Administrative Okrug,Khovrino District,55.869357,37.488795
2,Northern Administrative Okrug,Zapadnoye Degunino District,55.870548,37.520804
3,Northern Administrative Okrug,Timiryazevsky District,55.825817,37.557744
4,Northern Administrative Okrug,Koptevo District,55.830065,37.521738


In [16]:
def color_picker(Okrug):
    color = ['darkred', 'blue', 'green', 'lightblue', 'darkblue', \
             'lightgreen', 'white', 'orange', 'pink', 'purple', 'red', 'black']
    return(color[int((df[df['Okrug_Name'] == Okrug].index)[0])])


# Let's draw all districts on map
map_Moscow = folium.Map(location=[Moscow_latitude, Moscow_longitude], zoom_start=9)
for Okrug, District, lat, lng in zip(dff['Okrug'], \
                             dff['District'], \
                             dff['Latitude'], \
                             dff['Longitude']):
    label = '{}, {}'.format(Okrug, District)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color_picker(Okrug),
        fill=True,
        fill_color='#863100',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Moscow)  
map_Moscow

# Part 2 - Collecting data about New York

## 1. Download and Explore Dataset

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For your convenience, I downloaded the files and placed it on the server, so you can simply run a wget command and access the data. So let's go ahead and do that.


In [17]:
# Let's ccreate empty DataFrame to fill it with data in future
import urllib.request
url = 'https://cocl.us/new_york_dataset'
urllib.request.urlretrieve(url, 'newyork_data.json')

('newyork_data.json', <http.client.HTTPMessage at 0x1074e7b8>)

Next, let's load the data and tranform the data into a pandas dataframe

In [18]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods # Checking empty dataframe

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time. Then let's examine the resulting dataframe.

In [19]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

neighborhoods.head(3)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [20]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [35]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.

In [36]:
def color_pickerNY(borough):
    color = ['darkred', 'blue', 'green', 'black', 'darkblue']
    return(color[list(neighborhoods['Borough'].unique()).index(borough)])

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color_pickerNY(borough),
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

# Part 3 - comparison of two areas

### 1. let's pick one random borough from each city

In [38]:
NY_boroughs = list(neighborhoods['Borough'].unique())
Moscow_boroughs = list(dff['Okrug'].unique())
NY_rand_borough = NY_boroughs[random.randint(0, len(NY_boroughs))]
Moscow_rand_borough = Moscow_boroughs[random.randint(0, len(Moscow_boroughs))]
print(NY_rand_borough + '\n' + Moscow_rand_borough)

Manhattan
North-Western Administrative Okrug


Let's create two dataframes form selected boroughs

In [39]:
NY_rand_df = neighborhoods[neighborhoods['Borough'] == NY_rand_borough].reset_index(drop=True)
NY_rand_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [40]:
Moscow_rand_df = dff[dff['Okrug'] == Moscow_rand_borough].reset_index(drop=True)
Moscow_rand_df.head()

,Okrug,District,Latitude,Longitude
0,North-Western Administrative Okrug,Khoroshyovo-Mnyovniki District,55.778757,37.480074
1,North-Western Administrative Okrug,Shchukino District,55.800572,37.473055
2,North-Western Administrative Okrug,Strogino District,55.801335,37.403053
3,North-Western Administrative Okrug,Mitino District,55.844547,37.368606
4,North-Western Administrative Okrug,Kurkino District,55.891907,37.390106


### 2. Now let's draw selected boroughs on map

Getting coordinates of selected boroughs

In [69]:
NY_rand_coord = get_coord(NY_rand_borough)
Moscow_rand_coord = get_coord(Moscow_rand_borough)

In [70]:
map_Moscow_rand = folium.Map(location=[Moscow_rand_coord[0], Moscow_rand_coord[1]], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Moscow_rand_df['Latitude'], Moscow_rand_df['Longitude'], Moscow_rand_df['District']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Moscow_rand)  
    
map_Moscow_rand

In [71]:
map_NY_rand = folium.Map(location=[NY_rand_coord[0], NY_rand_coord[1]], zoom_start=11)

# add markers to map
for lat, lng, label in zip(NY_rand_df['Latitude'], NY_rand_df['Longitude'], NY_rand_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NY_rand)  
    
map_NY_rand

too few points on Moscow map... lets use postal offices

In [78]:
PO = pd.read_excel('PostOffices.xlsx', usecols = 'B,C,F,G')
PO.head()

,PostalCode,AdmArea,X_WGS84,Y_WGS84
0,101000,Central Administrative Okrug,37.637168,55.763861
1,123001,Central Administrative Okrug,37.588373,55.763037
2,119002,Central Administrative Okrug,37.587558,55.748001
3,109004,Central Administrative Okrug,37.662662,55.743727
4,105005,Central Administrative Okrug,37.677749,55.771255


In [84]:
map_Moscow_PO = folium.Map(location=[Moscow_latitude, Moscow_longitude], zoom_start=9)

# add markers to map
for lat, lng, label in zip(PO['Y_WGS84'], PO['X_WGS84'], PO['PostalCode']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        parse_html=False).add_to(map_Moscow_PO)  
    
map_Moscow_PO

In [134]:
Moscow_PO_boroughs = list(PO['AdmArea'].unique())
Moscow_PO_rand_borough = Moscow_PO_boroughs[random.randint(0, len(Moscow_PO_boroughs)-1)]
Moscow_rand_PO_df = PO[PO['AdmArea'] == Moscow_PO_rand_borough].reset_index(drop=True)
print(Moscow_PO_rand_borough)

North-Western Administrative Okrug


In [135]:
map_Moscow_PO_rand = folium.Map(location=[Moscow_rand_coord[0], Moscow_rand_coord[1]], zoom_start=10)

# add markers to map
for lat, lng, label in zip(Moscow_rand_PO_df['Y_WGS84'], Moscow_rand_PO_df['X_WGS84'], Moscow_rand_PO_df['PostalCode']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Moscow_PO_rand)  
    
map_Moscow_PO_rand